In [10]:
import os

In [11]:
%pwd

'd:\\software\\python_vs\\lone_approval_prediction'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\software\\python_vs\\lone_approval_prediction'

In [12]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/rppawar491/Lone_approval_prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "rppawar491"
os.environ["MLFLOW_TRACKING_PASSWARD"] = "024579b40f9cfcb8f0cb1be07a0ed82e8ae6f12e"

#### Entity

In [8]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class ModelEvaluationConfig:
    root_dir : Path
    x_test_data_path : Path
    y_test_data_path : Path
    model_path : Path
    all_params : dict
    metric_file_name : Path
    mlflow_uri : str


#### Configuration Manager

In [9]:
from lone_approval_prediction.constants import *
from lone_approval_prediction.utils.comman import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.logisticRegressor
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            x_test_data_path = config.x_test_data_path,
            y_test_data_path = config.y_test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            mlflow_uri = "https://dagshub.com/rppawar491/Lone_approval_prediction.mlflow"
        )
        
        return model_evaluation_config

#### Component